In [ ]:
# Uncomment the below command if you don't have openpyxl
#!pip install openpyxl 

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")

In [3]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = []
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['period'] = pd.Series(period)

In [4]:
market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])
market_data['date'] = market_data['Time (UTC+10)'].dt.date
columns_name = market_data.columns

# Change all column names into integer type for easier column calls
market_data.columns = [i for i in range(len(columns_name))]

for column in range(len(columns_name)):
    print(column, ":", columns_name[column])

0 : Time (UTC+10)
1 : Regions NSW Trading Price ($/MWh)
2 : Regions SA Trading Price ($/MWh)
3 : Regions TAS Trading Price ($/MWh)
4 : Regions VIC Trading Price ($/MWh)
5 : Regions NSW Trading Total Intermittent Generation (MW)
6 : Regions SA Trading Total Intermittent Generation (MW)
7 : Regions TAS Trading Total Intermittent Generation (MW)
8 : Regions VIC Trading Total Intermittent Generation (MW)
9 : Regions NSW Operational Demand (MW)
10 : Regions SA Operational Demand (MW)
11 : Regions TAS Operational Demand (MW)
12 : Regions VIC Operational Demand (MW)
13 : period
14 : date


In [5]:
###################################
# A function to find local minima and maxima of a given period given the data.
# A local minima can be defined as when the adjacent (previous and next) elements are
# greater than the current value. 
# Similarly, a local Maxima can be defined when the adjacent elements are smaller than
# the current value.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48
#      - period (default = 48) : preferable period with minimum of 48.
#
# Return:
#      - list of local minima, list of local maxima
#
# Created by: Gilbert
# Referenced from: https://www.geeksforgeeks.org/find-indices-of-all-local-maxima-and-local-minima-in-an-array/
###################################

def LocalMinMax(data, period = 48):
    # Set number of maximum discharge
    max_discharge_number = 5 # (5 * 5 = 25)
    max_charge_number = 5 # (5 * 4 = 20)
     
    len_data = len(data)
    
    local_max_id = [] 
    local_min_id = [] 
    
    # Change dataframe into numpy array
    spot_price = data[4].to_numpy()
    
    # Check the first element if it is local minima or maxima.
    # If first element is greater than the next, then it is 
    # local maxima and vice versa.
    if spot_price[0] > spot_price[1]:
        local_max_id.append(0)
    elif spot_price[0] < spot_price[1]:
        local_min_id.append(0)
    
    # Iterating over the second to the second to last element,
    # checking which element is local minima or maxima
    for i in range(1, len_data - 1):
        if(spot_price[i - 1] > spot_price[i] < spot_price[i + 1]): 
            local_min_id.append(i) 
        elif(spot_price[i-1] < spot_price[i] > spot_price[i + 1]): 
            local_max_id.append(i) 
    
    # Check the last element if it is local minima or maxima. 
    # If last element is greater than second to last, then it 
    # is local maxima and vice versa.
    if(spot_price[-1] > spot_price[-2]): 
        local_max_id.append(len_data - 1) 
    elif(spot_price[-1] < spot_price[-2]): 
        local_min_id.append(len_data - 1) 
    
    # Create a sorted dictionary of local maxima and minima
    local_max = {}
    local_min = {}
    for m in local_max_id:
        local_max[m] = spot_price[m]
    for m in local_min_id:
        local_min[m] = spot_price[m]
    
    sorted_local_max = sorted(local_max.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)
    sorted_local_min = sorted(local_min.items(), key = lambda kv:(kv[1], kv[0]))
    
    # Initialise empty array
    period_maxima = np.empty(len_data)
    period_maxima[:] = np.NaN
    
    period_minima = np.empty(len_data)
    period_minima[:] = np.NaN
    
    mod = len_data // period
    # Set the given local maxima index to a set length of period.
    # The maximum number of maxima set is 4 * (length of data // period).
    # For example, it can be assumed that if battery charge and discharge
    # is 1:1 ratio with a period of 48 and the time taken to charge is 5 
    # period and discharge is 4 period. Therefore, the possible maximum 
    # is 45 period in a day which are 5 times for each charge (5 * 5) and 
    # discharge (5 * 4) 

    for p_max in range(len(sorted_local_max)):
        period_maxima[sorted_local_max[p_max][0]] = p_max + 1
        if p_max + 1 == max_discharge_number * mod:
            break
        
    for p_min in range(len(sorted_local_min)):
        period_minima[sorted_local_min[p_min][0]] = p_min + 1
        if p_min + 1 == max_charge_number * mod:
            break
            
    return period_minima, period_maxima

In [6]:
def FindCombinations(data, period, point = 'minima'):
    if point == 'minima':
        combi_1 = (period, period + 1, period + 2, period + 3, period + 4)
        combi_2 = (period - 1, period, period + 1, period + 2, period + 3)
        combi_3 = (period - 2, period - 1, period, period + 1, period + 2)
        combi_4 = (period - 3, period - 2, period - 1, period, period + 1)
        combi_5 = (period - 4, period - 3, period - 2, period - 1, period)
        
        tmp = [combi_1, combi_2, combi_3, combi_4, combi_5]
        combinations = []
        for c in tmp:
            if c[0] > 0 and c not in combinations:
                if c[-1] <= 48:
                    combinations.append(c)
                    
    elif point == 'maxima':
        combi_1 = (period, period + 1, period + 2, period + 3)
        combi_2 = (period - 1, period, period + 1, period + 2)
        combi_3 = (period - 2, period - 1, period, period + 1)
        combi_4 = (period - 3, period - 2, period - 1, period)
        
        tmp = [combi_1, combi_2, combi_3, combi_4]
        combinations = []
        for c in tmp:
            if c[0] > 0 and c not in combinations:
                if c[-1] <= 48:
                    combinations.append(c)
            
    return combinations

In [7]:
def GetSpotPrice(data, combinations):
    spot_price = np.array(data[4])
    
    retrieved_price = []
    for combi in combinations:
        tmp = []
        for period in combi:
            tmp.append(spot_price[period - 1])
        retrieved_price.append(tmp)
    return np.array(retrieved_price)

In [8]:
def FindBestRevenue(data, period, point = 'minima', mlf = 0.991):
    combi = FindCombinations(data, period, point)
    spot_price = GetSpotPrice(data, combi)
    if point == 'minima':
        market_dispatch = np.array((150, 150, 150, 150, 44)) 
        revenue = (spot_price @ market_dispatch.T * (1 / mlf)) 
        revenue = revenue.tolist()
        best = revenue.index(min(revenue))
        return combi[best], min(revenue)
    elif point == 'maxima':
        market_dispatch = np.array((135, 135, 135, 117))
        revenue = (spot_price @ market_dispatch.T * (mlf)) 
        revenue = revenue.tolist()
        best = revenue.index(max(revenue))
        return combi[best], max(revenue)

### Testing

In [15]:
# TRAINING SET
# Training period is from 01/01/2018 to 30/06/2021 provided in the spec
cp3_start_period = '2020-07-17 00:30:00'
cp3_end_period   = '2020-07-18 00:00:00'

In [16]:
vic_spot_price = market_data[[0, 4, 13]]
cp_3 = vic_spot_price.loc[(vic_spot_price[0] >= cp3_start_period) & \
        (vic_spot_price[0] <= cp3_end_period)]

In [17]:
minima, maxima = LocalMinMax(cp_3)
print(minima)
print(maxima)

[nan nan nan nan nan nan nan  3. nan nan nan  5. nan nan nan nan nan nan
 nan nan nan nan nan  4. nan nan nan  1. nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan  2. nan nan]
[nan nan  5. nan nan nan nan nan nan nan nan nan nan nan nan nan  2. nan
 nan nan  3. nan  4. nan nan nan nan nan nan nan nan nan nan nan nan  1.
 nan nan nan nan nan nan nan nan nan nan nan nan]


In [18]:
cp_3

,0,4,13
44545,2020-07-17 00:30:00,75.51,2
44546,2020-07-17 01:00:00,73.98,3
44547,2020-07-17 01:30:00,75.57,4
44548,2020-07-17 02:00:00,71.94,5
44549,2020-07-17 02:30:00,74.10,6
44550,2020-07-17 03:00:00,67.36,7
44551,2020-07-17 03:30:00,58.04,8
44552,2020-07-17 04:00:00,51.85,9
44553,2020-07-17 04:30:00,74.53,10
44554,2020-07-17 05:00:00,67.32,11


In [20]:
minima_id = [(minima.tolist().index(i) + 1, i) for i in minima if not np.isnan(i)]
maxima_id = [(maxima.tolist().index(i) + 1, i) for i in maxima if not np.isnan(i)]

print(minima_id)
print(maxima_id)
revenues = []
for i in minima_id:
    curr_minima = i
    curr_min_combi, curr_min_rev = FindBestRevenue(cp_3, curr_minima[0], "minima")
    for m in maxima_id:
        if curr_minima[0] < m[0]:
            curr_max_combi, curr_max_rev = FindBestRevenue(cp_3, m[0], "maxima")
            revenues.append((curr_minima, m, curr_max_rev - curr_min_rev))

np.array(revenues, dtype = 'object')

# Idea to select highest revenue
# From discharge period select the highest, so in this case select (29, 1.0) and (37, 1.0) then remove pairs with (37, 1.0).
# Iterate over it until done.

[(8, 3.0), (12, 5.0), (24, 4.0), (28, 1.0), (46, 2.0)]
[(3, 5.0), (17, 2.0), (21, 3.0), (23, 4.0), (36, 1.0)]


array([[(8, 3.0), (17, 2.0), 12103.221023370337],
       [(8, 3.0), (21, 3.0), 3528.316043370338],
       [(8, 3.0), (23, 4.0), 1797.5840933703294],
       [(8, 3.0), (36, 1.0), 17567.535563370337],
       [(12, 5.0), (17, 2.0), 10532.605483511608],
       [(12, 5.0), (21, 3.0), 1957.700503511609],
       [(12, 5.0), (23, 4.0), 226.96855351160048],
       [(12, 5.0), (36, 1.0), 15996.920023511608],
       [(24, 4.0), (36, 1.0), 19233.610235418768],
       [(28, 1.0), (36, 1.0), 31660.270174873865]], dtype=object)

### Adding Local Minima and Maxima

In [14]:
# Selects Time, victoria spot price and period
vic_spot_price = market_data[[0, 4, 13]]

minima = []
maxima = []

period = 48

start = 0
end = period 
while end <= len(market_data):
    tmp_minima, tmp_maxima = LocalMinMax(vic_spot_price.iloc[start:end, :])
    
    minima.extend(tmp_minima)
    maxima.extend(tmp_maxima)
    start += period
    end += period

minima.append(float("nan"))
maxima.append(float("nan"))

market_data['minima'] = pd.Series(minima)
market_data['maxima'] = pd.Series(maxima)